In [ ]:
# default_exp paralel

# Pararel Downloading

This define a paralel executor for the downloader

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import time
from threading import Thread
from typing import List

from docarray import Document, DocumentArray
from rich.progress import Progress

from web2dataset.downloader import Downloader

In [ ]:
# export
class ParalelDownload:
    def __init__(
        self,
        path: str,
        downloader_cls: type[Downloader],
        num_workers: int,
        dataset_fn: str = "dataset.bin",
        *args,
        **kwargs,
    ):
        """Parallelize donwloading
        path: folder in which to save the files
        downloader_cls: the class of Downloader you want to parellize
        num_workers: the number of worker to use.
        dataset_fn: name of the file in which to save the docarray dataset, by default dataset.bin
        silence: to silence the logging and the progress bar
        *args: args to pass to the downloader
        *kwargs: kwargs to pass to the downloader
        """

        self.path = path
        self.dataset_fn = dataset_fn
        self.num_workers = num_workers
        self.downloaders = [
            downloader_cls(path=path, *args, **kwargs) for i in range(num_workers)
        ]

    def download(
        self,
        queries: List[str],
        n_item: int,
        silence: bool = False,
    ):
        with Progress(disable=silence) as progress:

            for i, _ in enumerate(self.downloaders):
                progress.add_task(f"Scrapping {queries[i]} ...", total=n_item)

            def task(downloader, task_id):
                while query := queries.pop() if queries else False:
                    downloader._download(query, n_item, progress, task_id)

            threads = []
            for task_id, downloader in enumerate(self.downloaders):
                threads.append(
                    Thread(
                        target=task,
                        args=(
                            downloader,
                            task_id,
                        ),
                    )
                )
                threads[-1].start()

            for t in threads:
                t.join()

            with open(f"{self.path}/{self.dataset_fn}", "wb") as f:
                self.docs = DocumentArray().empty()
                for d in self.downloaders:
                    self.docs.extend(d.docs)
                f.write(self.docs.to_bytes())

In [ ]:
test_folder = "/tmp/test_paralel"

In [ ]:
# hide
import shutil

shutil.rmtree(test_folder, ignore_errors=True)

let's define first a downloader to test

In [ ]:
import time


class BasicDownloader(Downloader):
    def _download(self, query: str, n_item: int, progress: Progress, task_id: int = 0):
        time.sleep(0.5)
        self.docs.extend(
            (Document(tag={"origin": "https://www.google.fr"}) for _ in range(n_item))
        )
        progress.update(task_id, advance=n_item)

In [ ]:
paralel_down = ParalelDownload(f"{test_folder}/my_search", BasicDownloader, 2)

In [ ]:
%%time
paralel_down.download(["a", "b", "c", "d"], 3)

Output()

CPU times: user 135 ms, sys: 3.23 ms, total: 138 ms
Wall time: 1.03 s


In [ ]:
assert len(paralel_down.docs) == 12